# Data Reading and Preprocessing


In [314]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
from itertools import combinations

In [315]:
df = pd.read_csv("Market_Basket_Optimisation_binarized.csv")
print(df.info())
df.head(3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7501 entries, 0 to 7500
Columns: 119 entries, almonds to zucchini
dtypes: int64(119)
memory usage: 6.8 MB
None


,almonds,antioxydant juice,asparagus,avocado,babies food,bacon,barbecue sauce,black tea,blueberries,body spray,...,turkey,vegetables mix,water spray,white wine,whole weat flour,whole wheat pasta,whole wheat rice,yams,yogurt cake,zucchini
0,1,1,0,1,0,0,0,0,0,0,...,0,1,0,0,1,0,0,1,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


**get_support** function evaluates the support value for a set given all the transactions.


In [316]:
def get_support(df: pd.DataFrame, itemset: set) -> float:

    if len(df) == 0:
        return 0.0

    return df.iloc[:, sorted(itemset)].all(axis=1).mean()



def test_get_support():
    # Create a DataFrame
    _df = pd.DataFrame(
        {
            "A": [1, 1, 0, 0, 1],
            "B": [1, 0, 1, 1, 0],
            "C": [0, 1, 0, 1, 1],
        }
    )

    # Test the function
    assert get_support(_df, {0}) == 0.6
    assert get_support(_df, {1}) == 0.6
    assert get_support(_df, {2}) == 0.6
    assert get_support(_df, {0, 1}) == 0.2
    assert get_support(_df, {0, 2}) == 0.4
    assert get_support(_df, {1, 2}) == 0.2
    assert get_support(_df, {0, 1, 2}) == 0.0

    print("All test cases passed!.")


# Run the test cases
# test_get_support()

In [317]:
import timeit

# Benchmarking using pandas-based function
itemset = {0, 1, 2}
num_iterations = 100


def get_support_without_pd(df, itemset):
    if len(df) == 0:
        return 0.0

    supports = []
    for index, row in df.iterrows():
        all_true = True
        for i in sorted(itemset):
            if not row[i]:
                all_true = False
                break
        if all_true:
            supports.append(1)
        else:
            supports.append(0)

    return sum(supports) / len(df)


time_taken_pd = timeit.timeit(lambda: get_support(df, itemset), number=num_iterations)

# Benchmarking using alternative function without pandas
time_taken_without_pd = timeit.timeit(
    lambda: get_support_without_pd(df, itemset), number=num_iterations
)

print(
    f"Time taken for {num_iterations} iterations (with pandas): {time_taken_pd} seconds"
)
print(
    f"Time taken for {num_iterations} iterations (without pandas): {time_taken_without_pd} seconds"
)

Time taken for 100 iterations (with pandas): 0.08205710000765976 seconds
Time taken for 100 iterations (without pandas): 25.229143800010206 seconds


In [318]:
def get_subsets(item_set):
    return [item_set - {item} for item in item_set]


def test_get_subsets():
    # Test case 1: Empty item set
    item_set_empty = set()
    expected_output_empty = []
    assert get_subsets(item_set_empty) == expected_output_empty

    # Test case 2: Non-empty item set
    item_set_non_empty = {1, 2, 3}
    expected_output_non_empty = [{2, 3}, {1, 3}, {1, 2}]
    assert get_subsets(
        item_set_non_empty) == expected_output_non_empty

    print("Test cases for get_subsets passed!")


test_get_subsets()

Test cases for get_subsets passed!


In [319]:
def is_valid_set(item_set, prev_level_sets):
    """ 
    Check if all the subsets of the item_set are present in the previous level sets.
    
    Parameters:
        item_set (list): The item set to be validated.
        prev_level_sets (list of lists): List of sets from the previous level.
        
    Returns:
        bool: True if all subsets of the item_set are present in prev_level_sets, False otherwise.
      
    Example:
        >>> prev_level_sets = [{1, 2}, {2, 3}, {1, 3}]
        >>> item_set = {1, 2, 4}
        in this example, a subset of item_set {1, 4} is not present in prev_level_sets.
        following the principle that if a set is not frequent, all its supersets are also not frequent,
        and since {1, 4} ie: not present in prev_level_sets, {1, 2, 4} is also not frequent.
    """
    if len(prev_level_sets) == 0:
        return False
    
    single_drop_subsets = get_subsets(item_set)
    for single_drop_set in single_drop_subsets:
        if single_drop_set not in prev_level_sets:
            return False
        
    return True


def test_is_valid_set():
    # Test case 1: Empty previous level sets
    prev_level_sets_empty = []
    item_set = {1, 2}
    assert is_valid_set(item_set, prev_level_sets_empty) is False

    # Test case 2: Item set not present in previous level sets
    prev_level_sets = [{3, 4}, {5, 6}]
    item_set_not_present = {1, 2}
    assert is_valid_set(item_set_not_present, prev_level_sets) is False

    # Test case 3: Item set present in previous level sets
    prev_level_sets_present = [{1, 2}, {2, 3}, {1, 3}, {3, 4}, {5, 6}]
    item_set_present = {1, 2, 3}
    assert is_valid_set(item_set_present, prev_level_sets_present) is True

    print("Test cases for is_valid_set passed!")


# Run the test cases
test_is_valid_set()

Test cases for is_valid_set passed!


**generate_candidate_item_sets** function generates the candidate item sets of size k from the frequent item sets of size k-1.


In [320]:
def generate_candidate_item_sets(frequent_item_sets, level):
    current_level_candidates = list()

    if len(frequent_item_sets[level - 1]) == 0:
        return current_level_candidates

    # Extract unique items from the frequent item sets of the previous level
    unique_items = set()
    prev_level_sets = list()
    for item_set, _ in frequent_item_sets[level - 1]:
        unique_items.update(item_set)
        prev_level_sets.append(item_set)

    # Generate candidates by combining unique items
    for candidate_set in combinations(unique_items, level + 1):
        # convert the tuple to a set
        candidate_set = set(candidate_set)
        # if the candidate set has a subset that doesn't exist in the frequent item sets of the previous level, skip it
        if is_valid_set(candidate_set, prev_level_sets):
            current_level_candidates.append(candidate_set)

    # calculate the support of each candidate set
    return current_level_candidates

**prune_candidates** function prunes the candidate sets evaluated based on the mean of the support for the current level.

In [321]:
def prune_candidates(df, current_level_candidates):
    post_prune_candidates_set = list()

    min_support = 0.0
    for candidate_set in current_level_candidates:
        support = get_support(df, candidate_set)
        min_support += support
        post_prune_candidates_set.append((candidate_set, support))

    min_support /= len(current_level_candidates)

    return [
        (candidate_set, support)
        for candidate_set, support in post_prune_candidates_set
        if support >= min_support
    ]


def test_prune_candidates():

    # Test case 1: Empty candidate set
    frequent_item_sets = {
        1: [({1}, 0.2), ({2}, 0.3), ({3}, 0.3)],
        2: [({1, 2}, 0.1), ({1, 3}, 0.2), ({2, 3}, 0.2), ({2, 4}, 0.3)],
    }
    level = 3
    candidate_set_empty = generate_candidate_item_sets(frequent_item_sets, level)
    # expected_output_empty = []
    # assert (
    #     prune_candidates(frequent_item_sets, level, candidate_set_empty)
    #     == expected_output_empty
    # )

    # # Test case 2: Candidate set with valid item sets
    # candidate_set_valid = [{1, 2, 3}, {2, 3, 4}, {3, 4, 5}]
    # expected_output_valid = [{1, 2, 3}, {2, 3, 4}, {3, 4, 5}]
    # assert (
    #     prune_candidates(frequent_item_sets, level, candidate_set_valid)
    #     == expected_output_valid
    # )

    # # Test case 3: Candidate set with invalid item sets
    # candidate_set_invalid = [{1, 2}, {2, 3}, {4, 5}]
    # expected_output_invalid = []
    # assert (
    #     prune_candidates(frequent_item_sets, level, candidate_set_invalid)
    #     == expected_output_invalid
    # )

    print("Test cases for pruning passed!")


test_prune_candidates()

Test cases for pruning passed!


## Main apriori algorithm


In [322]:
from collections import defaultdict


def apriori(df: pd.DataFrame):

    frequent_item_sets = defaultdict(list)

    print("level 0: ", end=" ")
    means = df.mean()
    min_support = means.mean()
    # first level candidates are the items themselves, filter them by min_support
    for i, support in enumerate(means[means >= min_support]):
        frequent_item_sets[0].append(({i}, support))

    print(f"{len(frequent_item_sets[0])} frequent items.")

    total_itemsets = 0
    for level in range(1, len(df.columns)):
        print(f"level {level}: ", end=" ")

        current_level_candidates = generate_candidate_item_sets(
            frequent_item_sets, level
        )

        if len(current_level_candidates) == 0:
            print(f"{level} reached, no more frequent item sets.")
            break
        else:
            print(f"{len(current_level_candidates)} candidates.", end=" ")

        frequent_item_sets[level] = prune_candidates(df, current_level_candidates)

        total_itemsets += len(frequent_item_sets[level])

        print(
            f"{len(frequent_item_sets[level])} frequent item sets, min support: {min_support}"
        )
    
    print(f"Total frequent item sets: {total_itemsets}")
    return frequent_item_sets

In [323]:
frequent_item_sets = apriori(df)

level 0:  32 frequent items.
level 1:  496 candidates. 

123 frequent item sets, min support: 0.03288973234941224
level 2:  299 candidates. 123 frequent item sets, min support: 0.03288973234941224
level 3:  50 candidates. 23 frequent item sets, min support: 0.03288973234941224
level 4:  1 candidates. 1 frequent item sets, min support: 0.03288973234941224
level 5:  5 reached, no more frequent item sets.
Total frequent item sets: 270
({0, 7}, 0.0009332089054792694)


In [329]:
import timeit

time_taken_apriori = timeit.timeit(lambda: apriori(df), number=num_iterations)

print(f"Time taken for {num_iterations} iterations of apriori: {time_taken_apriori} seconds")

level 0:  32 frequent items.
level 1:  496 candidates. 123 frequent item sets, min support: 0.03288973234941224
level 2:  299 candidates. 123 frequent item sets, min support: 0.03288973234941224
level 3:  50 candidates. 23 frequent item sets, min support: 0.03288973234941224
level 4:  1 candidates. 1 frequent item sets, min support: 0.03288973234941224
level 5:  5 reached, no more frequent item sets.
Total frequent item sets: 270
level 0:  32 frequent items.
level 1:  496 candidates. 123 frequent item sets, min support: 0.03288973234941224
level 2:  299 candidates. 123 frequent item sets, min support: 0.03288973234941224
level 3:  50 candidates. 23 frequent item sets, min support: 0.03288973234941224
level 4:  1 candidates. 1 frequent item sets, min support: 0.03288973234941224
level 5:  5 reached, no more frequent item sets.
Total frequent item sets: 270
level 0:  32 frequent items.
level 1:  496 candidates. 123 frequent item sets, min support: 0.03288973234941224
level 2:  299 candid

## Generating Association Rules

In [324]:
item_support_dict = dict()

item_list = list()
for level in frequent_item_sets:
    for set_support_pair in frequent_item_sets[level]:
        for i in set_support_pair[0]:
            item_list.append(df.columns[i])
        item_support_dict[frozenset(item_list)] = set_support_pair[1]
        item_list = list()

In [325]:
item_support_dict

{frozenset({'almonds'}): 0.03332888948140248,
 frozenset({'antioxydant juice'}): 0.03372883615517931,
 frozenset({'asparagus'}): 0.0871883748833489,
 frozenset({'avocado'}): 0.08105585921877083,
 frozenset({'babies food'}): 0.04679376083188908,
 frozenset({'bacon'}): 0.05999200106652446,
 frozenset({'barbecue sauce'}): 0.1638448206905746,
 frozenset({'black tea'}): 0.08038928142914278,
 frozenset({'blueberries'}): 0.0510598586855086,
 frozenset({'body spray'}): 0.17970937208372217,
 frozenset({'bramble'}): 0.0793227569657379,
 frozenset({'brownies'}): 0.1709105452606319,
 frozenset({'bug spray'}): 0.043060925209972005,
 frozenset({'burger sauce'}): 0.06332489001466471,
 frozenset({'burgers'}): 0.09532062391681109,
 frozenset({'butter'}): 0.0523930142647647,
 frozenset({'cake'}): 0.13211571790427942,
 frozenset({'candy bars'}): 0.09825356619117451,
 frozenset({'carrots'}): 0.04946007199040128,
 frozenset({'cauliflower'}): 0.047460338621517134,
 frozenset({'cereals'}): 0.0765231302493000

## association_rules 
generates the association rules in accordance with the given _minimum confidence_ value and the provided dictionary of itemsets against their support values. For itemsets of more than one element, it first finds all their subsets. For every subset A, it calculates the set B = itemset-A. If B is not empty, the confidence of B is calculated. If this value is more than _minimum confidence_ value, the rule _A->B_ is added to the list.


In [326]:
def association_rules(min_confidence, min_lift, support_dict) -> pd.DataFrame:
    rules_data = []

    for itemset, support in support_dict.items():

        if len(itemset) < 2:
            continue

        for i in range(1, len(itemset)):
            for A in combinations(itemset, i):
                A = frozenset(A)
                B = itemset - A

                support_A = support_dict[A]
                support_B = support_dict[B]

                confidence = support / support_A
                lift = support / (support_A * support_B)

                if confidence >= min_confidence and lift >= min_lift:
                    rules_data.append(
                        {"A": A, "B": B, "Confidence": confidence, "Lift": lift}
                    )

    rules_df = pd.DataFrame(rules_data)
    return rules_df

### Specify a minimum confidence and lift value to generate the association rules.

In [334]:
min_confidence = 0.5
min_lift = 0
rules = association_rules(
    min_confidence=min_confidence,
    min_lift=min_lift,
    support_dict=item_support_dict,
)

In [336]:
import timeit
# benchmarking the association_rules function
time_taken_association_rules = timeit.timeit(
    lambda: association_rules(min_confidence, min_lift, item_support_dict),
    number=num_iterations,
)

print(
    f"Time taken for {num_iterations} iterations of association_rules: {time_taken_association_rules} seconds"
)

Time taken for 100 iterations of association_rules: 0.14365209999959916 seconds


In [328]:
print("Number of rules: ", len(rules), "\n")
rules

Number of rules:  53 



,A,B,Confidence,Lift
0,"(almonds, carrots)",(burgers),0.571429,5.994805
1,"(almonds, chicken)",(burgers),0.555556,5.828283
2,"(almonds, cooking oil)",(chocolate),0.538462,5.664797
3,"(burgers, bug spray)",(chocolate),0.500000,5.260168
4,"(cake, cider)",(burgers),0.545455,5.722314
5,"(cooking oil, chili)",(chicken),0.714286,2.996564
6,"(chili, chicken)",(cooking oil),0.555556,8.885335
7,"(almonds, chicken, chocolate)",(burgers),0.500000,5.245455
8,"(cereals, almonds, chocolate)",(cake),0.666667,5.046081
9,"(cereals, almonds, cake)",(chocolate),0.666667,7.013558
